In [17]:
# Célula 1: Imports e Configuração de Caminhos
from urllib.parse import urlparse
import requests
import json
import os
from datetime import datetime, timezone
import tldextract # pip install tldextract se não tiver
from pathlib import Path
import sys

# --- Configuração de Caminhos ---
# Ajuste este caminho se o notebook não estiver na raiz ou em uma pasta 'notebooks' diretamente sob a raiz
try:
    # Tenta encontrar a raiz do projeto subindo a partir do local do notebook
    # Isso é útil se o notebook estiver em uma subpasta como 'notebooks'
    project_root = Path.cwd()
    while not (project_root / 'config').exists() or not (project_root / 'src').exists():
        if project_root.parent == project_root: # Chegou na raiz do sistema
            raise FileNotFoundError("Não foi possível localizar a raiz do projeto (pastas config/ e src/).")
        project_root = project_root.parent
except FileNotFoundError:
    # Fallback se a detecção automática falhar - defina manualmente
    print("Detecção automática da raiz do projeto falhou. Usando Path.cwd(). Verifique os caminhos.")
    project_root = Path.cwd() # Assume que o notebook está na raiz do projeto

print(f"Raiz do projeto detectada/definida como: {project_root}")

# Adiciona a raiz do projeto ao sys.path para permitir imports de 'config' e 'src'
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))
if str(project_root / 'src') not in sys.path: # Se 'config' estiver dentro de 'src', ajuste
     sys.path.insert(0, str(project_root / 'src'))


# Tenta importar o módulo settings
try:
    from config import settings
    print("Módulo config.settings carregado com sucesso.")
except ImportError as e:
    print(f"ERRO ao importar 'config.settings': {e}")
    print("Verifique se a raiz do projeto está correta e se 'config/settings.py' existe e é importável.")
    settings = None # Define para None para que o script possa continuar com avisos
except Exception as e:
    print(f"ERRO inesperado ao importar 'config.settings': {e}")
    settings = None

# --- Constantes ---
NEWSAPI_CONFIG_PATH = project_root / "config" / "newsapi_news_config.json"

print(f"Caminho para config da NewsAPI: {NEWSAPI_CONFIG_PATH}")

# Célula 2: Funções Utilitárias (simplificadas para o teste)

def load_json_file(file_path: Path):
    """Carrega um arquivo JSON."""
    if not file_path.exists():
        print(f"ERRO: Arquivo de configuração não encontrado: {file_path}")
        return None
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            return json.load(f)
    except json.JSONDecodeError as e:
        print(f"ERRO: Falha ao decodificar JSON de {file_path}: {e}")
        return None
    except Exception as e:
        print(f"ERRO inesperado ao ler {file_path}: {e}")
        return None

def get_domain_from_url(url: str) -> str | None:
    """Extrai o domínio registrável de uma URL."""
    if not url:
        return None
    try:
        extracted = tldextract.extract(url)
        return extracted.registered_domain if extracted.registered_domain else urlparse(url).hostname
    except Exception:
        return urlparse(url).hostname # Fallback simples
    
    # Célula 3: Função para Chamar a NewsAPI

def fetch_newsapi_data(api_key: str, base_url: str, query_name: str, params: dict):
    """Busca dados da NewsAPI para uma query específica."""
    if not api_key:
        print(f"[{query_name}] ERRO: API Key da NewsAPI não fornecida.")
        return None

    headers = {"X-Api-Key": api_key}
    print(f"[{query_name}] Fazendo requisição para: {base_url} com params: {params}")

    try:
        response = requests.get(base_url, headers=headers, params=params, timeout=20)
        response.raise_for_status()  # Levanta erro para status 4xx/5xx
        print(f"[{query_name}] Status da Resposta: {response.status_code}")
        return response.json()
    except requests.exceptions.HTTPError as http_err:
        print(f"[{query_name}] Erro HTTP: {http_err}")
        print(f"[{query_name}] Conteúdo da resposta (se houver): {response.text}")
    except requests.exceptions.ConnectionError as conn_err:
        print(f"[{query_name}] Erro de Conexão: {conn_err}")
    except requests.exceptions.Timeout as timeout_err:
        print(f"[{query_name}] Timeout: {timeout_err}")
    except requests.exceptions.RequestException as req_err:
        print(f"[{query_name}] Erro na Requisição: {req_err}")
    except json.JSONDecodeError as json_err:
        print(f"[{query_name}] Erro ao decodificar JSON da resposta: {json_err}. Resposta recebida: {response.text[:500]}")
    return None

# Célula 4: Carregar Configurações e Executar Teste (CORRIGIDA)

# Carregar API Key do settings
NEWSAPI_API_KEY = None
if settings:
    try:
        NEWSAPI_API_KEY = settings.NEWSAPI_API_KEY
        if NEWSAPI_API_KEY:
            print("API Key da NewsAPI carregada de settings.py.")
        else:
            print("ALERTA: NEWSAPI_API_KEY está vazia ou não definida em config.settings.")
    except AttributeError:
        print("ALERTA: NEWSAPI_API_KEY não encontrada como atributo em config.settings.")
else:
    print("ALERTA: Módulo config.settings não foi carregado. API Key não disponível.")


# Carregar configurações de query da NewsAPI
newsapi_config_data_wrapper = load_json_file(NEWSAPI_CONFIG_PATH) # Renomeado para clareza

# DEBUG Adicional (opcional, mas pode manter por enquanto)
print("\n--- DEBUGGING newsapi_config_data_wrapper ---")
if newsapi_config_data_wrapper is not None:
    print("Conteúdo de newsapi_config_data_wrapper (nível mais externo):")
    try:
        print(json.dumps(newsapi_config_data_wrapper, indent=2, ensure_ascii=False))
        if isinstance(newsapi_config_data_wrapper, dict):
            print(f"Chaves disponíveis no nível principal: {list(newsapi_config_data_wrapper.keys())}")
    except Exception as e:
        print(f"Erro ao formatar newsapi_config_data_wrapper: {e}")
else:
    print("newsapi_config_data_wrapper é None.")
print("--- FIM DO DEBUGGING ---\n")


newsapi_actual_config = None
if isinstance(newsapi_config_data_wrapper, dict) and "newsapi" in newsapi_config_data_wrapper:
    newsapi_actual_config = newsapi_config_data_wrapper["newsapi"] # <<<--- ACESSA O DICIONÁRIO INTERNO
    print("Configuração específica da 'newsapi' carregada com sucesso do wrapper.")
elif isinstance(newsapi_config_data_wrapper, dict) and "base_url" in newsapi_config_data_wrapper:
    # Caso o arquivo NÃO tenha a chave "newsapi" no topo e sim "base_url" diretamente (menos provável agora)
    newsapi_actual_config = newsapi_config_data_wrapper
    print("Configuração carregada diretamente (sem wrapper 'newsapi').")
else:
    print("ERRO: Estrutura esperada com a chave 'newsapi' (contendo 'base_url' e 'queries') não encontrada no arquivo de configuração.")


if NEWSAPI_API_KEY and newsapi_actual_config and isinstance(newsapi_actual_config, dict): # Verifica se é um dicionário
    newsapi_base_url = newsapi_actual_config.get("base_url")
    queries_to_test = newsapi_actual_config.get("queries", [])

    if not newsapi_base_url:
        print("ERRO: 'base_url' não encontrada DENTRO da configuração 'newsapi' do arquivo.")
    elif not queries_to_test:
        print("ALERTA: Nenhuma 'queries' encontrada DENTRO da configuração 'newsapi' para testar.")
    else:
        query_idx_to_test = 0 # Testar a primeira query
        if len(queries_to_test) > query_idx_to_test:
            query_config = queries_to_test[query_idx_to_test]
            query_name = query_config.get("query_name", f"Query de Teste {query_idx_to_test+1}")
            params = query_config.get("params", {})

            print(f"\n--- Iniciando teste para a query: '{query_name}' ---")
            api_response = fetch_newsapi_data(NEWSAPI_API_KEY, newsapi_base_url, query_name, params)

            if api_response:
                print(f"\n[{query_name}] Resposta da API recebida com sucesso!")
                print(f"  Status retornado pela API: {api_response.get('status')}")
                print(f"  Total de Resultados (conforme API): {api_response.get('totalResults')}")

                articles = api_response.get("articles", [])
                print(f"  Número de Artigos retornados nesta página: {len(articles)}")

                if articles:
                    print("\n--- Amostra de Artigos (primeiros 2 ou menos) ---")
                    for i, article in enumerate(articles[:2]): # Mostra os primeiros 2 artigos
                        print(f"\n  Artigo {i+1}:")
                        print(f"    Título: {article.get('title')}")
                        print(f"    Fonte (Nome API): {article.get('source', {}).get('name')}")
                        # ... (resto da impressão dos detalhes do artigo) ...
                    
                    print("\n--- Estrutura completa do PRIMEIRO artigo (JSON): ---")
                    print(json.dumps(articles[0], indent=4, ensure_ascii=False))
                else:
                    print(f"[{query_name}] Nenhum artigo foi retornado na resposta, embora o status da API seja OK.")
            else:
                print(f"[{query_name}] Nenhuma resposta válida recebida da API.")
        else:
            print(f"ERRO: Índice de query para teste ({query_idx_to_test}) fora do alcance. Existem {len(queries_to_test)} queries.")
elif not NEWSAPI_API_KEY:
    print("\nTeste não pode prosseguir: Chave da NewsAPI não está disponível.")
elif not newsapi_actual_config:
    print("\nTeste não pode prosseguir: Configuração da NewsAPI (contendo 'base_url' e 'queries') não carregada ou inválida.")

Raiz do projeto detectada/definida como: c:\ArgusAnalytics\ArgusAnalytics-Core
Módulo config.settings carregado com sucesso.
Caminho para config da NewsAPI: c:\ArgusAnalytics\ArgusAnalytics-Core\config\newsapi_news_config.json
API Key da NewsAPI carregada de settings.py.

--- DEBUGGING newsapi_config_data_wrapper ---
Conteúdo de newsapi_config_data_wrapper (nível mais externo):
{
  "newsapi": {
    "base_url": "https://newsapi.org/v2/everything",
    "queries": [
      {
        "query_name": "Petrobras Geral (NewsAPI)",
        "params": {
          "q": "Petrobras OR PETR4",
          "language": "pt",
          "sortBy": "publishedAt",
          "pageSize": 20
        },
        "target_company_ticker": "PETR4",
        "target_segment_name": null
      },
      {
        "query_name": "Setor O&G Brasil (NewsAPI)",
        "params": {
          "q": "(\"setor de petróleo\" OR \"óleo e gás\") AND Brasil NOT Petrobras NOT PETR4",
          "language": "pt",
          "sortBy": "publis